In [9]:
import getpass
import os
import dotenv
dotenv.load_dotenv()

# if not os.environ.get("OPENAI_API_KEY"):
#   os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter API key for OpenAI: ")

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini",openai_api_key=OPENAI_API_KEY)

In [15]:
from langchain_core.messages import HumanMessage
from langchain.memory import ConversationSummaryMemory, ConversationBufferMemory
from langgraph.graph import START, MessagesState, StateGraph

In [16]:
workflow = StateGraph(state_schema=MessagesState)

In [17]:
def call_model(state: MessagesState):
    response = llm.invoke(state['messages'])
    return {'messages': response}

In [18]:
workflow.add_edge(START, 'model')
workflow.add_node('model', call_model)

In [ ]:
memory = ConversationBufferMemory()
app = workflow.compile(checkpointer=memory)

In [26]:
from langchain.agents import load_tools, initialize_agent, AgentType
from langchain.chains import LLMMathChain
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

wikipedia = WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper())

llm_math = LLMMathChain.from_llm(llm=llm, verbose=True)

tools = load_tools(["wikipedia", "llm-math"], llm=llm)

agent = initialize_agent(
    tools,
    llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)

result = agent.run("What is the square root of the year Plato was born?")
print(result)

PydanticUserError: `LLMMathChain` is not fully defined; you should define `BaseCache`, then call `LLMMathChain.model_rebuild()`.

For further information visit https://errors.pydantic.dev/2.10/u/class-not-fully-defined